In [16]:
import nltk
import string
import jsonrpclib
import json
from nltk.corpus import wordnet as wn
import nltk.data
import re
from string import punctuation
from operator import itemgetter
import ast
from simplejson import loads
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA 
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
wordnet_lemmatizer = WordNetLemmatizer()
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
r = re.compile(r'[{}]'.format(punctuation))

name appears in the paragraph before


2 alternant lines 

if the name appears in the previous or following utterance with a pattern such as ',name,'

the object of the speech verb


In [20]:
f = open('annotation_speaker')
n = 0
instance = []
temp = []
speaker = []
p = 0
keyposition = []   #the linenum of the paragraph to be treated
for line in f:
    if n <= 29560:
        if '###############' in line:
            keyposition.append(p)
            current = [temp.pop()]
            previous = temp[:]
            temp = []
            continue
        if 'speaker:	' in line:
            speaker.append(line.split('speaker:	')[1].split('\r\n')[0])
            instance.append([current,previous,temp])
            temp = []
            p = 0
            continue
        if line.replace(' ','') != '\r\n':
            temp.append(line.replace('\r\n',''))
            p = p+1
    n = n+1
    

f.close()
#instance [0,1,2]:
#0 : current paragraph (including the object utterance)
#1 : 9 previous paragraphs
#2 : 5 following paragraphs

In [3]:
f = './result_parser.json'

def treatresultparser(f):
    f = open(f)
    result = []
    for line in f:
        if '#######seperator instance' in line:
            result.append([])
        elif '#######seperator cases' in line:
            result[-1].append([])
        elif '########seperator paragraphs' in line:
            result[-1][-1].append([])
        elif '{' in line:
            parsetree = []
            words = []
            dependency = []
            pos = [match.end() for match in re.finditer(re.escape('{"parsetree": "'), line)]
            pos2 = [match.start() for match in re.finditer(re.escape(')", "text"'), line)]
            pos3 = [match.end() for match in re.finditer(re.escape('"words": [['), line)]
            pos4 = [match.start() for match in re.finditer(re.escape('], "indexeddependencies":'), line)]
            pos5 = [match.end() for match in re.finditer(re.escape('"indexeddependencies": ['), line)]
            pos6 = [match.start() for match in re.finditer(re.escape('}, {"parsetree":'), line)]
            pos6.append(line.index(']}]}'))
            for i in range(len(pos)):
                parsetree.append(line[pos[i]:pos2[i]+1])
                words.append(line[pos3[i]:pos4[i]+1])
            for i in range(len(pos5)):
                dependency.append(line[pos5[i]:pos6[i]])
            result[-1][-1][-1].append(parsetree)
            result[-1][-1][-1].append(words)
            for i in range(len(dependency)):
                if len(dependency[i]) > 2:
                    if dependency[i][-2:] == ']]' :
                        dependency[i] = dependency[i][:-1]
                if dependency[i] == ']':
                    dependency[i] = '[]'
            result[-1][-1][-1].append(dependency)
    f.close()
    return result

result = treatresultparser(f)

In [2]:
def treatresultparser_new(f):
    f = open(f)
    result = []
    for line in f:
        if '#######seperator' in line:
            result.append([])
        elif '{' in line:
            parsetree = []
            words = []
            dependency = []
            pos = [match.end() for match in re.finditer(re.escape('{"parsetree": "'), line)]
            pos2 = [match.start() for match in re.finditer(re.escape(')", "text"'), line)]
            pos3 = [match.end() for match in re.finditer(re.escape('"words": [['), line)]
            pos4 = [match.start() for match in re.finditer(re.escape('], "indexeddependencies":'), line)]
            pos5 = [match.end() for match in re.finditer(re.escape('"indexeddependencies": ['), line)]
            pos6 = [match.start() for match in re.finditer(re.escape('}, {"parsetree":'), line)]
            pos6.append(line.index(']}]}'))
            for i in range(len(pos)):
                parsetree.append(line[pos[i]:pos2[i]+1])
                words.append(line[pos3[i]:pos4[i]+1])
            for i in range(len(pos5)):
                dependency.append(line[pos5[i]:pos6[i]])
            result[-1].append(parsetree)
            result[-1].append(words)
            for i in range(len(dependency)):
                if len(dependency[i]) > 2:
                    if dependency[i][-2:] == ']]' :
                        dependency[i] = dependency[i][:-1]
                if dependency[i] == ']':
                    dependency[i] = '[]'
            result[-1].append(dependency)
    f.close()
    return result

In [5]:
def countwordperline(instance,num_instance,keyposition):
    numlist = []
    for i in range(3):
        for j in range(len(instance[num_instance][i])):
            new_strs = r.sub(' ',instance[num_instance][i][j])
            num = len(new_strs.split())
            numlist.append(num)
    numlist.insert(keyposition[num_instance]-1, numlist.pop(0))
    return numlist
            
countwordperline(instance,62,keyposition)

[29, 7, 6, 16, 83, 9, 25, 3, 42, 53, 82, 62, 143, 109, 35]

In [6]:
def convertoffset(numlist,linenum,offset,sentence):
    temp = sentence[0:offset]
    new_strs = r.sub(' ',temp)
    num = len(new_strs.split())
    pos = 0
    for i in range(linenum-1):
        pos = pos + numlist[i]
    return pos+num+1

convertoffset(countwordperline(instance,0,keyposition),keyposition[0],33,instance[0][0][0])

676

# no use, we can get the index from the loop number
def convertposition(numlist,linenum,position): 
    temp = position
    i = 0 
    while temp > 0:
        temp = temp - numlist[i]
        i = i+1
    i = i-1
    if i == linenum-1:
        return 0,0
    elif i>linenum-1:
        return 2,i-linenum
    else:
        return 1,i

convertposition(countwordperline(instance,0),keyposition[0],669)

def infodependency(name,offset,ni,nc,np):
    new_strs = r.sub(' ',instance[ni][nc][np])
    num = len(new_strs.split())
    part = tokenizer.tokenize(instance[ni][nc][np])
    length = []
    t = len(part)-1
    i = 0
    while i <= t:
        if len(part[i]) > 3:
            if part[i][-3:] == '...' and i < t:
                part[i] = part[i] +' '+ part[i+1]
                part.pop(i+1)
                i -= 1
                t -= 1
                continue
            if part[i][:3] == 'And':
                part[i-1] = part[i-1] +' '+ part[i]
                part.pop(i)
                i -= 1
                t -= 1
                continue
        i += 1
    for i in range(len(part)):
        if i == 0:
            length.append(len(part[i]))
        elif i > 0:
            part[i] = ' '+part[i]
            length.append(len(part[i]))

    s = 0
    linenum = 0
    offset_new = 0
    for i in range(len(length)):
        s += length[i]
        if offset - s < 0:
            linenum = i
            offset_new = offset - length[i-1]
            break
    key = name+'-'+str(len(re.findall(r"[\w]+|[^\s\w]", part[linenum][:offset_new]))+1)
    temp = ast.literal_eval(result[ni][nc][np][2][linenum])
    head = []
    for obj in temp:
        if key in obj:
            if obj[0] == 'det' or obj[0] == 'amod':
                head.append(obj[2].split('-'))
    head.sort(key=itemgetter(1))
    complete = ''
    for obj in head:
        complete += obj[0]+' '
    complete += name
    return complete
    
infodependency('lady',49,116,1,3)

In [7]:
#this offset global contains newline
def convertoffsetglobal(offsetlocal,ni,nc,nl,instance):
    if nc == 0:
        for k in range(len(instance[ni][1])):
            offsetlocal += len(instance[ni][1][k]) +1
    elif nc == 1:
        for k in range(nl):
            offsetlocal += len(instance[ni][1][k]) +1
    elif nc == 2:
        for k in range(len(instance[ni][1])):
            offsetlocal += len(instance[ni][1][k]) +1
        for k in range(nl):
            offsetlocal += len(instance[ni][2][k]) +1
        offsetlocal += len(instance[ni][0][0]) +1
    return offsetlocal

In [8]:
def ner(instance,result,keyposition):
    entities = []
    n = 0
    for obj in instance:
        numlist = countwordperline(instance,n,keyposition)
        current = [[]]
        temp = []
        for z in range(len(result[n][0][0][0])):
            temp = temp + result[n][0][0][0][z].split('] [')
        for frag in temp:
            if 'PartOfSpeech=NNP' in frag or 'NamedEntityTag=PERSON' in frag:
                text = frag.split('Text=')[1].split(' ')[0]
                position = int(frag.split('CharacterOffsetBegin=')[1].split(' ')[0])
                offset = convertoffsetglobal(position,n,0,0,instance)
                pos = convertoffset(numlist,keyposition[n],position,obj[0][0])
                current[0].append([text,position,pos,offset,'PERSON'])
            elif 'PartOfSpeech=N' in frag:
                text = frag.split('Text=')[1].split(' ')[0]
                position = int(frag.split('CharacterOffsetBegin=')[1].split(' ')[0])
                offset = convertoffsetglobal(position,n,0,0,instance)
                try:
                    pos = convertoffset(numlist,keyposition[n],position,obj[0][0])
                except IndexError:
                    print 
                    ccccccc
                lex = []
                for synset in wn.synsets(text):
                    lex.append(synset.lexname())
                lenlex = 0
                for c in lex:
                    if 'noun' in c:
                        lenlex += 1
                if lex == []:
                    current[0].append([text,position,pos,offset,'NOUN'])
                elif lenlex == 0:
                    continue
                elif float(lex.count('noun.person'))/float(lenlex) >= 0.4:
                    current[0].append([text,position,pos,offset,'NOUN'])
        temp = []
        for z in range(len(result[n][0][0][1])):
            temp = temp + result[n][0][0][1][z].split('] [')
        for frag in temp:
            part = frag.split('"')
            if part[part.index('PartOfSpeech')+2] == 'NNP' or part[part.index('NamedEntityTag')+2] == 'PERSON':
                text = frag.split('"')[1]
                position = int(part[part.index('CharacterOffsetBegin')+2])
                offset = convertoffsetglobal(position,n,0,0,instance)
                pos = convertoffset(numlist,keyposition[n],position,obj[0][0])
                current[0].append([text,position,pos,offset,'PERSON'])
            elif 'N' in part[part.index('PartOfSpeech')+2]:
                text = frag.split('"')[1]
                position = int(part[part.index('CharacterOffsetBegin')+2])
                offset = convertoffsetglobal(position,n,0,0,instance)
                pos = convertoffset(numlist,keyposition[n],position,obj[0][0])
                lex = []
                for synset in wn.synsets(text):
                    lex.append(synset.lexname())
                lenlex = 0
                for c in lex:
                    if 'noun' in c:
                        lenlex += 1
                if lex == []:
                    current[0].append([text,position,pos,offset,'NOUN'])
                elif lenlex == 0:
                    continue
                elif float(lex.count('noun.person'))/float(lenlex) >= 0.4:
                    current[0].append([text,position,pos,offset,'NOUN'])
        previous = []
        for i in range(len(obj[1])):
            previous.append([])
            temp = []
            for x in range(len(result[n][1][i][0])):
                try:
                    temp = temp + result[n][1][i][0][x].split('] [')
                except IndexError:
                    continue
            for frag in temp:
                if 'PartOfSpeech=NNP' in frag or 'NamedEntityTag=PERSON' in frag:
                    text = frag.split('Text=')[1].split(' ')[0]
                    position = int(frag.split('CharacterOffsetBegin=')[1].split(' ')[0])
                    offset = convertoffsetglobal(position,n,1,i,instance)
                    pos = convertoffset(numlist,i+1,position,obj[1][i])
                    previous[i].append([text,position,pos,offset,'PERSON'])
                elif 'PartOfSpeech=N' in frag:
                    text = frag.split('Text=')[1].split(' ')[0]
                    position = int(frag.split('CharacterOffsetBegin=')[1].split(' ')[0])
                    offset = convertoffsetglobal(position,n,1,i,instance)
                    pos = convertoffset(numlist,i+1,position,obj[1][i])
                    lex = []
                    for synset in wn.synsets(text):
                        lex.append(synset.lexname())
                    lenlex = 0
                    for c in lex:
                        if 'noun' in c:
                            lenlex += 1
                    if lex == []:
                        previous[i].append([text,position,pos,offset,'NOUN'])
                    elif lenlex == 0:
                        continue
                    elif float(lex.count('noun.person'))/float(lenlex) >= 0.4:
                        previous[i].append([text,position,pos,offset,'NOUN'])
            temp = []
            for x in range(len(result[n][1][i][1])):
                temp = temp + result[n][1][i][1][x].split('] [')
            for frag in temp:
                part = frag.split('"')
                if part[part.index('PartOfSpeech')+2] == 'NNP' or part[part.index('NamedEntityTag')+2] == 'PERSON':
                    text = frag.split('"')[1]
                    position = int(part[part.index('CharacterOffsetBegin')+2])
                    offset = convertoffsetglobal(position,n,1,i,instance)
                    pos = convertoffset(numlist,i+1,position,obj[1][i])
                    previous[i].append([text,position,pos,offset,'PERSON'])
                elif 'N' in part[part.index('PartOfSpeech')+2]:
                    text = frag.split('"')[1]
                    position = int(part[part.index('CharacterOffsetBegin')+2])
                    offset = convertoffsetglobal(position,n,1,i,instance)
                    pos = convertoffset(numlist,i+1,position,obj[1][i])
                    lex = []
                    for synset in wn.synsets(text):
                        lex.append(synset.lexname())
                    lenlex = 0
                    for c in lex:
                        if 'noun' in c:
                            lenlex += 1
                    if lex == []:
                        previous[i].append([text,position,pos,offset,'NOUN'])
                    elif lenlex == 0:
                        continue
                    elif float(lex.count('noun.person'))/float(lenlex) >= 0.4:
                        previous[i].append([text,position,pos,offset,'NOUN'])
        following = []
        for j in range(len(obj[2])):
            following.append([])
            temp = []
            for y in range(len(result[n][2][j][0])):
                try:
                    temp = temp + result[n][2][j][0][y].split('] [')
                except IndexError:
                    continue
            for frag in temp:
                if 'PartOfSpeech=NNP' in frag or 'NamedEntityTag=PERSON' in frag:
                    text = frag.split('Text=')[1].split(' ')[0]
                    position = int(frag.split('CharacterOffsetBegin=')[1].split(' ')[0])
                    offset = convertoffsetglobal(position,n,2,j,instance)
                    pos = convertoffset(numlist,keyposition[n]+j+1,position,obj[2][j])
                    following[j].append([text,position,pos,offset,'PERSON'])
                elif 'PartOfSpeech=N' in frag:
                    text = frag.split('Text=')[1].split(' ')[0]
                    position = int(frag.split('CharacterOffsetBegin=')[1].split(' ')[0])
                    offset = convertoffsetglobal(position,n,2,j,instance)
                    pos = convertoffset(numlist,keyposition[n]+j+1,position,obj[2][j])
                    lex = []
                    for synset in wn.synsets(text):
                        lex.append(synset.lexname())
                    lenlex = 0
                    for c in lex:
                        if 'noun' in c:
                            lenlex += 1
                    if lex == []:
                        following[j].append([text,position,pos,offset,'NOUN'])
                    elif lenlex == 0:
                        continue
                    elif float(lex.count('noun.person'))/float(lenlex) >= 0.4:
                        following[j].append([text,position,pos,offset,'NOUN'])
            temp = []
            for y in range(len(result[n][2][j][1])):
                temp = temp + result[n][2][j][1][y].split('] [')
            for frag in temp:
                part = frag.split('"')
                if part[part.index('PartOfSpeech')+2] == 'NNP' or part[part.index('NamedEntityTag')+2] == 'PERSON':
                    text = frag.split('"')[1]
                    position = int(part[part.index('CharacterOffsetBegin')+2])
                    offset = convertoffsetglobal(position,n,2,j,instance)
                    pos = convertoffset(numlist,keyposition[n]+j+1,position,obj[2][j])
                    following[j].append([text,position,pos,offset,'PERSON'])
                elif 'N' in part[part.index('PartOfSpeech')+2]:
                    text = frag.split('"')[1]
                    position = int(part[part.index('CharacterOffsetBegin')+2])
                    offset = convertoffsetglobal(position,n,2,j,instance)
                    pos = convertoffset(numlist,keyposition[n]+j+1,position,obj[2][j])
                    lex = []
                    for synset in wn.synsets(text):
                        lex.append(synset.lexname())
                    lenlex = 0
                    for c in lex:
                        if 'noun' in c:
                            lenlex += 1
                    if lex == []:
                        following[j].append([text,position,pos,offset,'NOUN'])
                    elif lenlex == 0:
                        continue
                    elif float(lex.count('noun.person'))/float(len(lex)) >= 0.4:
                        following[j].append([text,position,pos,offset,'NOUN'])
        entities.append([current,previous,following])
        n = n +1
    return entities

entities = ner(instance,result,keyposition)
for i in range(len(entities)):
    for j in range(3):
        for k in range(len(entities[i][j])):
            entities[i][j][k].sort(key=itemgetter(2))
entities_new = entities[:]

#entities [name, offset in the paragraph, word position in the instance]

In [9]:
#combine contiguous name entities and separate those within and outside quotation marks
def contiguous(instance,namelist,ni,cls,nl):
    i = 0
    t = len(namelist)-1
    newlist = {}
    newlist['in'] = []
    newlist['out'] = []
    while i < t:
        if namelist[i+1][1]-namelist[i][1] == len(namelist[i][0])+1 and instance[ni][cls][nl][namelist[i][1]+len(namelist[i][0])] == ' ' and namelist[i+1][4] == namelist[i][4]:
            namelist[i][0] = namelist[i][0]+' '+namelist[i+1][0]
            namelist.pop(i+1)
            t = t - 1
            i = i - 1
        i = i + 1
    for obj in namelist:
        if instance[ni][cls][nl][0:obj[1]].count('"')%2 == 1:
            newlist['in'].append(obj)
        else:
            newlist['out'].append(obj)
    return newlist

for i in range(len(entities)):
    for j in range(3):
        for k in range(len(instance[i][j])):
            entities_new[i][j][k] = contiguous(instance,entities[i][j][k],i,j,k)

In [10]:
def findentities_out(entities_new):
    entities_out = []
    entities_in = []
    for i in range(len(entities_new)):
        entities_out.append([])
        entities_in.append([])
        for j in range(3):
            entities_out[i].append([])
            entities_in[i].append([])
            for k in range(len(entities_new[i][j])):
                if entities_new[i][j][k] == {}:
                    entities_out[i][j].append([])
                    entities_in[i][j].append([])
                else:
                    entities_out[i][j].append([])
                    entities_out[i][j][k] = entities_new[i][j][k]['out']
                    entities_in[i][j].append([])
                    entities_in[i][j][k] = entities_new[i][j][k]['in']
    return entities_out,entities_in

entities_out,entities_in = findentities_out(entities_new)

In [ ]:
#####only for noting who is the speaker from the candidates
i = 0
#print instance[i]
fact = []
for i in range(len(instance)):
    print instance[i][0][0]
    for j in range(3):
        for k in range(len(entities_out[i][j])):
            for l in range(len(entities_out[i][j][k])):
                print entities_out[i][j][k][l], str(j), str(k), str(l) + '\n'
    fact.append(input("which : "))
#print result[i][0][0][2][1]
print fact
####maybe I should add those adj and det and see if it improve the result######
#####The Tattered Prince#####
#####the High Septon######

In [11]:
f = open('./fact')
fact = []
for line in f:
    fact = ast.literal_eval(line)
f.close()

###is there any quote in the paragraph which this person shows
###how many quotes between this person and the target quote
###all the index of paragraphs in which this person shows
###the last puctuation marks of the last quote and is there a close quotation mark
###whether this is a name entity or a personal noun
###whether this person is a object of a speech verb

notfound = []
for i in range(len(entities_out)):
    found = 0
    for j in range(3):
        for k in range(len(entities_out[i][j])):
            for obj in entities_out[i][j][k]:
                if speaker[i] == obj[0]:
                    found = 1
    if found == 0:
        notfound.append(i)
print notfound

In [12]:
def countpunctuation(sentence,l):
    punctuation = [',', '.', '\n', '!', '?', '"']
    for i in range(len(punctuation)):
        l[i] = l[i] + sentence.count(punctuation[i])

In [13]:
# to extract the distance between the candidat and the quote, with counting the punctuations between them
def distance(instance,entities,num_instance,keyposition):
    sentence = instance[num_instance][0][0]
    numlist = countwordperline(instance,num_instance,keyposition)
    if sentence.count('"') == 1:
        offset = [sentence.index('"')]
        position = [convertoffset(countwordperline(instance,num_instance,keyposition),keyposition[num_instance],offset[0],instance[num_instance][0][0])]
    else:
        offset = []
        p = 0
        for c in sentence:
            if c == '"':
                offset.append(p)
            p = p+1
        position = []
        for i in range(2):
            position.append(convertoffset(countwordperline(instance,num_instance,keyposition),keyposition[num_instance],offset[i],instance[num_instance][0][0]))
    dist = []
    cp = []
    for i in range(3):
        for j in range(len(entities[num_instance][i])):
            for k in range(len(entities[num_instance][i][j])):
                cp.append([0,0,0,0,0,0])
                pos = entities[num_instance][i][j][k][2]
                ofs = entities[num_instance][i][j][k][1]
                if len(position) == 1:
                    dist.append([abs(pos-position[0])])
                else:
                    if pos <= position[0]:
                        dist.append([position[0]-pos])
                    elif pos >= position[1]:
                        dist.append([pos-position[1]])
                stc = instance[num_instance][i][j]
                if len(offset)>1:
                    if i == 1:
                        countpunctuation(stc[ofs:],cp[-1])
                        cp[-1][2] = cp[-1][2]+1
                        for s in range(j+1,keyposition[num_instance]-1):
                            cp[-1][2] = cp[-1][2]+1
                            countpunctuation(instance[num_instance][i][s],cp[-1])
                        countpunctuation(sentence[:offset[0]],cp[-1])
                    elif (i == 0 and ofs < offset[0]):
                        countpunctuation(sentence[ofs:offset[0]],cp[-1])
                    elif i == 2:
                        countpunctuation(stc[:ofs],cp[-1])
                        for s in range(j):
                            countpunctuation(instance[num_instance][i][s],cp[-1])
                            cp[-1][2] = cp[-1][2]+1
                        countpunctuation(sentence[offset[1]+1:],cp[-1])
                        cp[-1][2] = cp[-1][2]+1
                    else:
                        countpunctuation(sentence[offset[1]+1:ofs+1],cp[-1])
                else:
                    if i == 1:    
                        countpunctuation(stc[ofs:],cp[-1])
                        cp[-1][2] = cp[-1][2]+1
                        for s in range(j+1,keyposition[num_instance]-1):
                            cp[-1][2] = cp[-1][2]+1
                            countpunctuation(instance[num_instance][i][s],cp[-1])
                        countpunctuation(sentence[:offset[0]],cp[-1])
                    elif i == 2:
                        countpunctuation(stc[:ofs],cp[-1])
                        cp[-1][2] = cp[-1][2]+1
                        for s in range(j):
                            cp[-1][2] = cp[-1][2]+1
                            countpunctuation(instance[num_instance][i][s],cp[-1])
                    else:
                        countpunctuation(sentence[ofs:offset[0]],cp[-1])
                        
    return dist,cp

dst = []
feature = []
order = []
for i in range(len(entities_out)):
    dst.append(distance(instance,entities_out,i,keyposition))
    feature.append(dst[i][0][:])
    order.append(feature[i][:])
    temp = sorted(range(len(order[i])), key=lambda k: order[i][k])
    n = 0
    for obj in temp:
        order[i][obj].append(n)
        n += 1

In [14]:
def detailparagraph(instance,num_instance,entities,keyposition):
    dp = []
    keyline = keyposition[num_instance]
    offset = 10 - keyline
    numlist = countwordperline(instance,num_instance,keyposition)
    for i in range(45):
        dp.append(0)
    for i in range(3):
        for j in range(len(entities[num_instance][i])):
            num = instance[num_instance][i][j].count('"')
            num = num/2+num%2
            if i == 1:
                dp[3*(j+offset)+1] += num
            elif i == 0:
                dp[28] += num
            elif i == 2:
                dp[30+3*j+1] += num
            for obj in entities[num_instance][i][j]:
                if i == 1:
                    dp[3*(j+offset)] += 1
                elif i == 0:
                    dp[27] += 1
                elif i == 2:
                    dp[30+3*j] += 1
    for i in range(len(numlist)):
        dp[2+3*(i+offset)] = numlist[i]
    return dp
                
dp = []
for i in range(len(entities_out)):
    dp.append(detailparagraph(instance,i,entities_out,keyposition))

In [15]:
def findsimilarname(entities_i):
    names = []
    for i in range(3):
        for j in range(len(entities_i[i])):
            for obj in entities_i[i][j]:
                names.append(obj+[i,j])
                
    for obj in names:
        if obj[0].find(' ') == -1:
            bad = 0
            for obj2 in names:
                if obj[0].lower() in obj2[0].lower():
                    for obj3 in names:
                        if obj[0].lower() in obj3[0].lower():
                            part2 = obj2[0].lower().split(' ')
                            part3 = obj3[0].lower().split(' ')
                            not_match = 0
                            for t in part2:
                                if t not in part3 and part2.index(t) != 0:
                                    not_match = 1
                            for t in part3:
                                if t not in part2 and not_match == 1 and part3.index(t) != 0:
                                    bad = 1
            if bad == 1:
                names.remove(obj)         
                            
    
    names_new = []
    for obj in names:
        temp = obj[0].split(' ')
        for obj2 in names:
            if obj != obj2:
                for t in temp:
                    if ' '+t.lower()+' ' in ' '+obj2[0].lower()+' ' and ((temp.index(t) != 0  and len(temp) > 1) or (len(temp) == 1)):
                        tmp = obj2[0].lower().split(' ')
                        if len(temp) == 2 and len(tmp) == 2 and (t.lower() == tmp[1] and tmp[0] not in [x.lower() for x in temp]):
                            continue
                        else:
                            found = 0
                            for i in range(len(names_new)):
                                if obj in names_new[i] and obj2 not in names_new[i]:
                                    names_new[i].append(obj2)
                                    found = 1
                                elif obj2 in names_new[i] and obj not in names_new[i]:
                                    names_new[i].append(obj)
                                    found = 1
                                elif obj2 in names_new[i] and obj in names_new[i]:
                                    found = 1
                                    pass
                            if found == 0:
                                names_new.append([obj,obj2])
    for obj in names_new:
        for obj2 in obj:
            for obj3 in names_new:
                if obj != obj3:
                    if obj2 in obj3:
                        if len(obj) >= len(obj3):
                            names_new.remove(obj3)
                        else:
                            try:
                                names_new.remove(obj)
                            except ValueError:
                                pass
    return names_new

In [16]:
def nameoccurrence(entities,keyposition):
    nocc = []
    for k in range(len(entities)):
        nocc.append([])
        names_new = findsimilarname(entities[k])
        key = keyposition[k]
        filling = 10 - key
        for m in range(len(names_new)):
            pl = []
            for l in range(15):
                pl.append(0)
            for obj2 in names_new[m]:
                if obj2[-2] == 0:
                    pl[9] += 1
                elif obj2[-2] == 1:
                    pl[obj2[-1]+filling] += 1
                elif obj2[-2] == 2:
                    pl[10+obj2[-1]] += 1
            names_new[m].append(pl)

        for i in range(3):
            for j in range(len(entities[k][i])):
                for obj in entities[k][i][j]:
                    num = 1
                    for obj2 in names_new:
                        if obj+[i,j] in obj2:
                            num = len(obj2)-1
                            res = [num]+obj2[-1]
                    if num == 1:
                        pl = []
                        for l in range(15):
                            pl.append(0)
                        if i == 0:
                            pl[9] += 1
                        elif i == 1:
                            pl[j+filling] += 1
                        elif i == 2:
                            pl[10+j] += 1
                        res = [num]+pl
                    nocc[-1].append(res)
    return nocc

nocc = nameoccurrence(entities_out,keyposition)

In [17]:
def transformresult(result_new,position,instance):
    result = []
    for i in range(len(instance)):
        result.append([])
        for j in range(3):
            result[-1].append([])
            for k in range(len(instance[i][j])):
                result[-1][-1].append([])
                if j == 0:
                    result[-1][-1][-1] = result_new[position[i]]
                elif j == 1:
                    result[-1][-1][-1] = result_new[position[i]-(len(instance[i][j])-k)]
                elif j == 2:
                    result[-1][-1][-1] = result_new[position[i]+k+1]
    return result

In [18]:
#### the next part is for treating the target text ####
f = open('./target_text')
instance_target = []
temp = []
p = 0
keyposition_target = []
line_position = []
for line in f:
    if '###############' in line:
        keyposition_target.append(p)
        current = [temp.pop()]
        previous = temp[:]
        temp = []
        line_position.append(int(line.split('\r\n')[0].replace('#','')))
        continue
    if 'speaker:' in line:
        instance_target.append([current,previous,temp])
        temp = []
        p = 0
        continue
    if line.replace(' ','') != '\r\n':
        temp.append(line.replace('\r\n',''))
        p = p+1
    


f.close()

result_target_new = treatresultparser_new('./result_parser_target_new.json')
result_target = transformresult(result_target_new,line_position,instance_target)
entities_target = ner(instance_target,result_target,keyposition_target)
for i in range(len(entities_target)):
    for j in range(3):
        for k in range(len(entities_target[i][j])):
            entities_target[i][j][k].sort(key=itemgetter(2))
entities_target_new = entities_target[:]
for i in range(len(entities_target)):
    for j in range(3):
        for k in range(len(instance_target[i][j])):
            entities_target_new[i][j][k] = contiguous(instance_target,entities_target[i][j][k],i,j,k)
entities_target_out,entities_target_in = findentities_out(entities_target_new)

In [19]:
#word,CharacterOffsetBegin,PartOfSpeech,NamedEntityTag,Lemma,case,line
def analyseparser(result,num_instance,instance,entities):
    POS = []
    for i in [1,0,2]:
        for j in range(len(result[num_instance][i])):
            for k in range(len(result[num_instance][i][j][1])):
                temp = []
                entity = result[num_instance][i][j][1][k].split('"')[1]
                info = ast.literal_eval(result[num_instance][i][j][1][k].split(', ',1)[1].replace(']',''))
                offset = convertoffsetglobal(int(info['CharacterOffsetBegin']),num_instance,i,j,instance)
                temp = [entity,offset,info['PartOfSpeech'],info['NamedEntityTag'],info['Lemma']]
                POS.append(temp)
                temp = []
                part = result[num_instance][i][j][0][k].split('] [')
                for l in range(len(part)):
                    info = part[l].split(' ')
                    entity = ''
                    CharacterOffsetBegin = ''
                    PartOfSpeech = ''
                    Lemma = ''
                    NamedEntityTag = ''
                    for obj in info:
                        if '[Text=' in obj:
                            entity = obj[6:]
                        elif 'Text=' in obj:
                            entity = obj[5:]
                        if 'CharacterOffsetBegin=' in obj:
                            CharacterOffsetBegin = int(obj[21:])
                        if 'PartOfSpeech=' in obj:
                            PartOfSpeech = obj[13:]
                        if 'Lemma=' in obj:
                            Lemma = obj[6:]
                        if 'NamedEntityTag=' in obj:
                            NamedEntityTag = obj[15:]
                    if CharacterOffsetBegin == '':
                        continue
                    offset = convertoffsetglobal(CharacterOffsetBegin,num_instance,i,j,instance)
                    temp = [entity,offset,PartOfSpeech,NamedEntityTag,Lemma]
                    POS.append(temp)
    return POS

#feature: 5 words near the candidate: 1 expression verb, 2 punctuation mark, 3 person, 4 newline, 5 nothing, 0 other cases
def neighbourhood(instance,num_instance,entities,result):
    f = open('./speechverbs')
    speechverb = []
    for line in f:
        if line != '\n':
            speechverb.append(line.split('\n')[0].lower())
    DISTANCE = 5
    text = ''
    nb = []
    for i in [1,0,2]:
        for obj in instance[num_instance][i]:
            text += obj+'\n'
    analyse = analyseparser(result,num_instance,instance,entities)
    for i in range(3):
        for j in range(len(entities[num_instance][i])):
            for obj in entities[num_instance][i][j]:
                temp = []
                offset = obj[1]
                offset = convertoffsetglobal(offset,num_instance,i,j,instance)
                list1 = [(m.group(0), m.start(0)) for m in re.finditer(r"[\w]+|[.]+|[\w]+[-][\w]+|["+string.punctuation+"]|[\n]", text[:offset])]
                list2 = [(m.group(0), m.start(0)+offset) for m in re.finditer(r"[\w]+|[.]+|[\w]+[-][\w]+|["+string.punctuation+"]|[\n]", text[offset:])] 
                if len(list1) >= DISTANCE:
                    previous = list1[-DISTANCE:]
                elif len(list1) < DISTANCE:
                    previous = list1[-len(list1):]
                if len(list2) > DISTANCE:
                    following = list2[1:DISTANCE+1]
                elif len(list2) <= DISTANCE:
                    following = list2[1:len(list2)]
                filling = DISTANCE-len(previous)
                for k in range(filling):
                    temp.append(5)
                for part in previous + following:
                    writen = 0
                    if part[0] == '\n':
                        temp.append(4)
                        writen = 1
                    num = 0
                    for c in part[0]:
                        if c in string.punctuation:
                            num += 1
                    if num == len(part[0]) and writen != 1:
                        temp.append(2)
                        writen = 1
                    for elem in analyse:
                        if elem[1] == part[1] and elem[0] == part[0]:
                            if writen != 1 and (elem[2] == 'NNP' or elem[3] == 'PERSON'):
                                temp.append(3)
                                writen = 1
                            if writen != 1 and ('VB' in elem[2] and elem[4].lower() in speechverb):
                                temp.append(1)
                                writen = 1
                        elif elem[1] == part[1] and elem[0] != part[0]:
                            pass
                    if writen == 0:
                        temp.append(0)
                nb.append(temp)
                filling = DISTANCE-len(following)
                for k in range(filling):
                    temp.append(5)
    f.close()
    return nb

#nb = []
#for i in range(len(instance)):            
#    nb.append(neighbourhood(instance,i,entities_out,result))

In [21]:
def countentities(instance,entities):
    ce = []
    for i in range(len(entities)):
        ce.append([])
        offset_quote = convertoffsetglobal(instance[i][0][0].index('"'),i,0,0,instance)
        for j in range(3):
            for k in range(len(entities[i][j])):
                for obj in entities[i][j][k]:
                    num = 0
                    offset = obj[3]
                    ce[-1].append([])
                    for l in range(3):
                        for m in range(len(entities[i][l])):
                            for obj2 in entities[i][l][m]:
                                if (obj2[3]-offset)*(obj2[3]-offset_quote) <0:
                                    num += 1
                    ce[-1][-1].append(num)
    return ce

ce = countentities(instance,entities_out)

preprocessing: need to combine those quotes with multilines

In [22]:
### for function tokenize
def findcase(token,pos,name_similar):
    if '<C' in token:
        sublist = [int(token[2:-1]),pos]
        for obj in name_similar:
            temp = 1
            for elem in [x in name_similar for x in sublist]:
                temp *= elem
            if temp == 1:
                return 1
        return 2
    elif '<N' in token:
        return 3
    elif '<P' in token:
        return 4
    elif '<Q' in token:
        return 5
    elif '<TQ' in token:
        return 6
    elif '<S' in token:
        return 7

In [23]:
def tokenize(instance,entities,result):
    f = open('./speechverbs')
    speechverb = []
    for line in f:
        if line != '\n':
            speechverb.append(line.split('\n')[0].lower())
    exclude = set(string.punctuation)
    tk = []
    NUM_TOKENS = 3   # number of tokens taken both sides
    for i in range(len(instance)):
        text = ''
        num = 0
        ofs = []
        temp = 0
        for j in [1,0,2]:
            for k in range(len(instance[i][j])):
                text += instance[i][j][k]+'\n'
                for obj in entities[i][j][k]:
                    temp += len(obj[0])-len('<C'+str(num)+'>')
                    ofs.append([temp,obj[3]])
                    num += 1
        quote_start = convertoffsetglobal(instance[i][0][0].index('"'),i,0,0,instance)
        quote_end = min(text[quote_start+1:].index('\n'),text[quote_start+1:].index('"'))+quote_start+2
        analyse = analyseparser(result,i,instance,entities)
        sv = []
        for obj in analyse:
            if 'V' in obj[2] and obj[4].lower() in speechverb and (int(obj[1]) > quote_end or int(obj[1]) < quote_start):
                sv.append(obj)
        text = text[:quote_start]+'<TQ>'+text[quote_end:]
        offset = quote_end - quote_start - len('<TQ>')
        ofs.append([offset,quote_start])
        ofs = sorted(ofs,key=itemgetter(1))
        for t in range(ofs.index([offset,quote_start])+1,len(ofs)):
            ofs[t][0] += offset
        ofs.remove([offset,quote_start])
        for j in [2,0,1]:
            for k in reversed(range(len(entities[i][j]))):
                for obj in reversed(entities[i][j][k]):
                    name_start = obj[3]
                    name_end = obj[3] + len(obj[0])
                    if name_start > quote_start:
                        name_start -= offset
                        name_end -= offset
                    text = text[:name_start]+'<C'+str(num-1)+'>'+text[name_end:]
                    num -= 1
        pos = []
        for obj in sv:
            filling = 0
            for j in range(len(ofs)):
                if obj[1] < ofs[0][1]:
                    if obj[1] > quote_end:
                        verb_start = obj[1] - offset
                        verb_end = obj[1]+len(obj[0]) - offset
                        pos.append((verb_start,verb_end))
                        break
                    else:
                        verb_start = obj[1]
                        verb_end = obj[1]+len(obj[0])
                        pos.append((verb_start,verb_end))
                        break
                elif obj[1] < ofs[j][1]:
                    if ofs[j][1] > quote_end and ofs[j-1][1] < quote_start and obj[1] > quote_end:
                        verb_start = obj[1]-ofs[j-1][0] - offset
                        verb_end = obj[1]-ofs[j-1][0]+len(obj[0]) - offset
                        pos.append((verb_start,verb_end))
                        break
                    else:
                        verb_start = obj[1]-ofs[j-1][0]
                        verb_end = obj[1]-ofs[j-1][0]+len(obj[0])
                        pos.append((verb_start,verb_end))
                        break
        for obj in reversed(pos):
            text = text[:obj[0]]+'<S>'+text[obj[1]:]
        text = text.replace('\n','<N>')
        pos = []
        num = 0
        locked = 0
        mark = 0
        for c in text:
            if c == '"' and locked == 0:
                locked = 1
                start = num
            elif c == '"' and locked == 1:
                locked = 0
                end = num
                pos.append((start,end))
            elif c == '<' and locked == 1:
                mark = 1
            elif c == 'N' and mark == 1:
                end = num
                locked = 0
                mark = 0
                pos.append((start,end))
            elif mark == 1:
                mark = 0
            num += 1
        for obj in reversed(pos):
            text = text[:obj[0]]+'<Q>'+text[obj[1]+1:]
        for c in string.punctuation:
            if c != '<' and c != '>' and c != '.':
                text=text.replace(c,'<P>')
        pos = []
        num = 0
        locked = 0
        for c in text:
            if c == '<' and locked == 0:
                locked = 1
                temp = num
            elif c == '<' and locked == 1:
                pos.append(temp)
            elif c == '>' and locked == 1:
                locked = 0
            elif c == '>' and locked == 0:
                pos.append(num)
            num += 1
        for obj in reversed(pos):
            text = text[:obj]+'<P>'+text[obj+1:]
        text = re.sub(r"[.]+", '<P>', text)
        name_similar = findsimilarname(entities[i])
        order = []
        for j in [1,0,2]:
            for obj in entities[i][j]:
                order += [x[3] for x in obj]
        for j in range(len(name_similar)):
            for k in range(len(name_similar[j])):
                name_similar[j][k] = order.index(name_similar[j][k][3])
        locked = 0
        tokens = ''
        for ch in text:
            if ch == '<':
                temp = ''
                locked = 1
            elif ch == '>':
                locked = 0
                tokens += '<'+temp+'> '
            elif locked == 1:
                temp += ch
        tokens = tokens.split()
        tk.append([])
        for j in range(3):
            for k in range(len(entities[i][j])):
                for obj in entities[i][j][k]:
                    temp = []
                    pos = order.index(obj[3])
                    for l in range(len(tokens)):
                        if '<C'+str(pos)+'>' in tokens[l]:
                            if l < NUM_TOKENS:
                                filling = NUM_TOKENS - l
                                for m in range(filling):
                                    temp.append(0)
                                for m in range(l):
                                    temp.append(findcase(tokens[m],pos,name_similar))
                                for m in range(l+1,l+NUM_TOKENS+1):
                                    temp.append(findcase(tokens[m],pos,name_similar))
                            elif l+NUM_TOKENS >= len(tokens):
                                filling = l+NUM_TOKENS-len(tokens)+1
                                for m in range(l-3,l):
                                    temp.append(findcase(tokens[m],pos,name_similar))
                                for m in range(l+1,len(tokens)):
                                    temp.append(findcase(tokens[m],pos,name_similar))
                                for m in range(filling):
                                    temp.append(0)
                            elif l < NUM_TOKENS and l+NUM_TOKENS >= len(tokens):
                                filling1 = NUM_TOKENS - l
                                filling2 = l+NUM_TOKENS-len(tokens)+1
                                for m in range(filling1):
                                    temp.append(0)
                                for m in range(l):
                                    temp.append(findcase(tokens[m],pos,name_similar))
                                for m in range(l+1,len(tokens)):
                                    temp.append(findcase(tokens[m],pos,name_similar))
                                for m in range(filling2):
                                    temp.append(0)
                            else:
                                for m in range(l-3,l):
                                    temp.append(findcase(tokens[m],pos,name_similar))
                                for m in range(l+1,l+NUM_TOKENS+1):
                                    temp.append(findcase(tokens[m],pos,name_similar))
                    tk[-1].append(temp)
    f.close()
    return tk
tk = tokenize(instance,entities_out,result)
#### <CN> candidate, <N> newline, <P> punctuation, <Q> quote, <TQ> target quote, <S> speech verb
#### 0: nothing, 1: the same candidate, 2: other candidates, 3: newline, 4: punctuation, 5: other quote, 6: target quote, 7: speech verb

In [24]:
###[1 one quotation mark or not, how many quote in it, 1 whether speaker is mentioned or not,1 whether other speaker is mentioned or not, how far from the start, the length in word]
def quotefeature(instance,entities_out,entities_in,mode):
    qf = []
    if mode == 0:
        p_0 = 0
        p_1 = 0
    elif mode == 1:
        p_0 = 1
        p_1 = -1
    elif mode == 2:
        p_0 = 2
        p_1 = 0
    for i in range(len(instance)):
        qf.append([])
        if len(instance[i][p_0]) == 0 or '"' not in instance[i][p_0][p_1]:
            for j in range(3):
                for k in range(len(entities_out[i][j])):
                    for obj in entities_out[i][j][k]:
                        qf[-1].append([0,0,0,0,0,0])
            continue
        if instance[i][p_0][p_1].count('"') == 1:
            num_qm = 1
            num_quote = 1
        else:
            num_quote = instance[i][p_0][p_1].count('"')%2 + instance[i][p_0][p_1].count('"')/2
            num_qm = 0
        text = instance[i][p_0][p_1]
        text_temp = re.split(r'[^0-9A-Za-z\']+',text[:text.index('"')])
        while '' in text_temp:
            text_temp.remove('')
        distance = len(text_temp)
        quote = ''
        part = instance[i][p_0][p_1].split('"')
        for p in range(len(part)):
            if p%2 == 1:
                quote += part[p]
        text_temp = re.split(r'[^0-9A-Za-z\']+',quote)
        while '' in text_temp:
            text_temp.remove('')
        length = len(text_temp)
        temp_qf = []
        entities_mixed = []
        entities_extended = []
        for j in range(3):
            entities_mixed.append([])
            for k in range(len(entities_out[i][j])):
                entities_mixed[-1].append([])
                entities_mixed[-1][-1] = []
                for obj in entities_out[i][j][k]:
                    entities_extended.append(obj)
                    entities_mixed[-1][-1].append(obj+['out'])
            if j == 0 and k == 0:
                temp = []
                for obj in entities_in[i][j][k]:
                    temp.append(obj+['in'])
                entities_mixed[j][k] += temp
        namegroup = findsimilarname(entities_mixed)
        samein = []
        for obj in namegroup:
            samein.append([])
            for obj2 in obj:
                if obj2[5] == 'in':
                    for obj3 in obj:
                        for obj4 in entities_extended:
                            mt = 0
                            for l in range(4):
                                if obj4[l] == obj3[l]:
                                    mt += 1
                            if mt == 4:
                                samein[-1].append(obj4)
        while [] in samein:
            samein.remove([])
        for j in range(3):
            for k in range(len(entities_out[i][j])):
                for obj in entities_out[i][j][k]:
                    found = 0
                    found_else = 0
                    for obj2 in namegroup:
                        for obj3 in obj2:
                            match = 0
                            for l in range(4):
                                if obj[l] == obj3[l]:
                                    match += 1
                            if match == 4:
                                for obj4 in obj2:
                                    if obj4[5] == 'in':
                                        found = 1
                    rest = len(samein)-found
                    if rest == 0:
                        found_else = 0
                    else:
                        found_else = 1
                    temp_qf = [num_qm,num_quote,found,found_else,distance,length]
                    qf[-1].append(temp_qf)
    return qf

qf = quotefeature(instance,entities_out,entities_in,0)
qf_1 = quotefeature(instance,entities_out,entities_in,1)
qf_2 = quotefeature(instance,entities_out,entities_in,2)
for i in range(len(qf)):
    for j in range(len(qf[i])):
        qf[i][j] += qf_1[i][j] + qf_2[i][j]

In [25]:
for i in range(len(feature)):
    for j in range(len(feature[i])):
        feature[i][j] += dst[i][1][j]
        feature[i][j] += dp[i]
        #feature[i][j] += nb[i][j]
        feature[i][j] += ce[i][j]
        feature[i][j] += nocc[i][j]
        feature[i][j] += tk[i][j]
        feature[i][j] += qf[i][j]

In [26]:
position_fact = []
for i in range(len(entities_out)):
    pos = 0
    if fact[i][0] == 0:
        pos = fact[i][2]
        position_fact.append(pos)
    elif fact[i][0] == 1:
        pos += len(entities_out[i][0][0])
        for j in range(fact[i][1]):
            pos += len(entities_out[i][1][j])
        pos += fact[i][2]
        position_fact.append(pos)
    elif fact[i][0] == 2:
        pos += len(entities_out[i][0][0])
        for j in range(len(entities_out[i][1])):
            pos += len(entities_out[i][1][j])
        for j in range(fact[i][1]):
            pos += len(entities_out[i][2][j])
        pos += fact[i][2]
        position_fact.append(pos)

In [27]:
X = []
for obj in feature:
    X += obj
Y = []
for i in range(len(feature)):
    temp = []
    for j in range(len(feature[i])):
        if j == position_fact[i]:
            temp.append(1)
        else:
            temp.append(0)
    Y += temp
clf = RandomForestClassifier(n_estimators=250)
clf.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
def findname(index,num_instance):
    counter = 0
    temp_i = 0
    temp_j = 0
    temp_counter = 0
    for i in range(3):
        for j in range(len(entities_target_out[num_instance][i])):
            temp_counter = counter
            counter += len(entities_target_out[num_instance][i][j])
            temp_j = j
            temp_i = i
            if index-counter < 0:
                offset = index-temp_counter
                return temp_i,temp_j,offset

In [29]:
dst_target = []
feature_target = []
order_target = []
for i in range(len(entities_target_out)):
    dst_target.append(distance(instance_target,entities_target_out,i,keyposition_target))
    feature_target.append(dst_target[i][0][:])
    order_target.append(feature_target[i][:])
    temp = sorted(range(len(order_target[i])), key=lambda k: order_target[i][k])
    n = 0
    for obj in temp:
        order_target[i][obj].append(n)
        n += 1
dp_target = []
for i in range(len(entities_target_out)):
    dp_target.append(detailparagraph(instance_target,i,entities_target_out,keyposition_target))
nb_target = []
#for i in range(len(instance_target)):            
#    nb_target.append(neighbourhood(instance_target,i,entities_target_out,result_target))
ce_target = []
ce_target = countentities(instance_target,entities_target_out)
nocc_target = []
nocc_target = nameoccurrence(entities_target_out,keyposition_target)
tk_target = []
tk_target = tokenize(instance_target,entities_target_out,result_target)
qf_target = []
qf_target = quotefeature(instance_target,entities_target_out,entities_target_in,0)
qf_target_1 = quotefeature(instance_target,entities_target_out,entities_target_in,1)
qf_target_2 = quotefeature(instance_target,entities_target_out,entities_target_in,2)
for i in range(len(qf_target)):
    for j in range(len(qf_target[i])):
        qf_target[i][j] += qf_target_1[i][j] + qf_target_2[i][j]
for i in range(len(feature_target)):
    for j in range(len(feature_target[i])):
        feature_target[i][j] += dst_target[i][1][j]
        feature_target[i][j] += dp_target[i]
        #feature_target[i][j] += nb_target[i][j]
        feature_target[i][j] += ce_target[i][j]
        feature_target[i][j] += nocc_target[i][j]
        feature_target[i][j] += tk_target[i][j]
        feature_target[i][j] += qf_target[i][j]

In [38]:
res = []
for t in range(len(feature_target)):
    temp = []
    for i in range(len(feature_target[t])):
        nc,nl,n = findname(i,t)
        temp.append([nc,nl,n,entities_target_out[t][nc][nl][n],clf.predict_proba([feature_target[t][i]])[0][1].item()])
    res.append(sorted(temp,key=itemgetter(4))[-1])

In [39]:
with open('./result.spk', 'w') as fp:
    for obj in res:
        json.dump(obj,fp)
        fp.write('\n')

In [59]:
print float(res.count(1))/len(res)    ### random forest 250

0.839382940109


In [36]:
print float(res.count(1))/len(res)   ### ramdon forest 100

0.81941923775


In [55]:
print float(res.count(1))/len(res)   ### Ada boost

0.638838475499


In [29]:
print float(res.count(1))/len(res)   ### Decision Tree  5

0.790381125227


In [ ]:
#feature [1 distance in words from the quote, 
#         1 the ordinal position, 
#         6 numbers of ',', '.', '\n', '!', '?', '"' between name and quote,
#         45 Number of names, quotes and words in each one of 15 paragraphs
#         10 words of the neighbourhood (unused)
#         1 number of entities between name and quote
#         1 total number of appearance of this entity
#         15 numbers of appearance of this entity in each paragraph
#         6 numbers of tokens 3 each side of the target]

#         1 this name is the subject or object of speech verb
#         9 speakers of 9 previous paragraphs 
#         10-fold cross validation(to be continued)

In [3]:
# part 2  coreference resolution
# require to load function treatresultparser_new
f = './result_parser_target_new.json'
result_target_new = treatresultparser_new(f)
keyword = ['mother','father','son','daughter','uncle','aunt','child','brother','sister'
          ,'cousin','wife','husband','niece','nephew','grandfather','grandmother']
# his/her/its... ,adj,  keyword, name 

In [4]:
result_token = []
for obj in result_target_new:
    result_token.append([])
    for i in range(2):
        if i == 1:
            for j in range(len(obj[i])):
                temp = ast.literal_eval(obj[i][j].split(', ',1)[1][:-2])
                result_token[-1].append([obj[i][j].split(', ',1)[0].replace('"',''),int(temp['CharacterOffsetBegin']),int(temp['CharacterOffsetEnd']),temp['PartOfSpeech']])
        if i == 0:
            for j in range(len(obj[i])):
                if '] [' in obj[i][j]:
                    temp = obj[i][j].split('] [')
                    for k in range(len(temp)):
                        result_token[-1].append([temp[k].split('Text=')[1].split(' ')[0],int(temp[k].split('CharacterOffsetBegin=')[1].split(' ')[0]),int(temp[k].split('CharacterOffsetEnd=')[1].split(' ')[0]),temp[k].split('PartOfSpeech=')[1].split(' ')[0]])

for i in range(len(result_token)):
    result_token[i] = sorted(result_token[i],key=itemgetter(1))

In [5]:
#print result_token[5]
info_pattern = []
for i in range(len(result_token)):
    info_pattern.append([])
    temp = ''
    stat = 0
    for obj2 in result_token[i]:
        if obj2[3] == 'PRP$' and stat == 0:
            stat = 1
            temp += obj2[0]+' '
        elif obj2[3] == 'JJ' and stat != 0:
            temp += obj2[0] +' '
        elif obj2[0].lower() in keyword and stat == 1:
            stat = 2
            temp += obj2[0]+' '
        elif 'NNP' in obj2[3] and stat == 2:
            temp += obj2[0]+' '
            info_pattern[-1].append(temp)
            stat = 3
        elif 'NNP' in obj2[3] and stat == 3:
            temp += obj2[0]+' '
            info_pattern[-1].pop()
            info_pattern[-1].append(temp)
        else:
            temp = ''
            stat = 0

In [6]:
f = open('./text')
raw_text = []
for line in f:
    if line.replace(' ','')!='\r\n' and line!='\n':
        raw_text.append(line)
f.close()

In [7]:
text_coref = []
for i in range(len(info_pattern)):
    if info_pattern[i] != []:
        for j in range(len(info_pattern[i])):
            temp = ''
            if i < 2:
                for t in range(i+1):
                    temp += raw_text[t]
                temp = temp.replace('\r\n','')
            elif i >= 2:
                for t in [-2,-1,0]:
                    temp += raw_text[i+t]
                temp = temp.replace('\r\n','')
            if '"' in temp:
                temp2 = []
                for m in re.finditer('"', temp):
                     temp2.append(m.start())
                pos = temp.index(info_pattern[i][j][:-1])
                for t in range(len(temp2)):
                    if pos < temp2[t]:
                        pos2 = t-1
                        break
                    pos2 = t
                temp3 = temp.split('"')
                temp = ''
                if pos2%2 == 1:
                    for k in range(len(temp3)):
                        if k%2 == 0:
                            temp += temp3[k]
                elif pos2%2 == 0:
                    if pos2 == len(temp2)-1:
                        for k in range(len(temp3)):
                            if k%2 == 0:
                                temp += temp3[k]
                        temp += temp3[-1]
                    else:
                        for k in range(len(temp3)):
                            if k%2 == 0:
                                temp += temp3[k]
                            if k == pos2+1:
                                temp += '"'+temp3[k]+'"'
            text_coref.append(temp)

In [8]:
info_pattern_new = []
pattern_line = []
n = 0
for obj in info_pattern:
    if obj != []:
        for i in range(len(obj)):
            pattern_line.append(n)
            if obj[i].endswith(' '):
                obj[i] = obj[i][:-1]
            info_pattern_new.append(obj[i])
    n += 1

In [9]:
info_parser = []
server = jsonrpclib.Server("http://localhost:8080")
for obj in text_coref:
    info_parser.append(loads(server.parse(obj)))
info_coref = []
for obj in info_parser:
    try:
        info_coref.append(obj['coref'])
    except KeyError:
        info_coref.append([])

In [10]:
def analysecoreference(text_coref,info_coref,info_pattern_new):
    result = []
    for i in range(len(text_coref)):
        num_sent = 0
        num_token = 0
        temp = tokenizer.tokenize(text_coref[i])
        for p in reversed(range(len(temp))):
            if '.  ' in temp[p] and temp[p][temp[p].index('.  ')-1]!='.':
                stock = temp[p].split('.  ',1)
                temp[p] = stock[0]+'.'
                temp.insert(p+1,stock[1])
            elif temp[p].endswith('...') and p!= len(temp)-1:
                temp[p] += '  '+temp[p+1]
                temp.pop(p+1)
        for j in range(len(temp)):
            if info_pattern_new[i] in temp[j]:
                num_sent = j
                temp2 = word_tokenize(temp[j])
                pattern_split = word_tokenize(info_pattern_new[i])
                pattern_new = ''
                for obj in pattern_split:
                    pattern_new += obj
                for p in range(len(temp2)-2):
                    t = temp2[p] + temp2[p+1] + temp2[p+2]
                    if t in pattern_new:
                        num_token = p
                        break
        for obj in info_coref[i]:
            for obj2 in obj:
                for obj3 in obj2:
                    if word_tokenize(info_pattern_new[i])[0] == obj3[0] and obj3[1] == num_sent and obj3[3] == num_token:
                        result.append([obj,pattern_line[i],i])
    return result

result_coref = analysecoreference(text_coref,info_coref,info_pattern_new)

In [11]:
relation_object = []
for obj in info_pattern_new:
    relation = ''
    position_relation = 0
    o = ''
    temp = word_tokenize(obj)
    for i in range(len(temp)):
        if temp[i] in keyword:
            position_relation = i
            relation = temp[i]
    for j in range(position_relation+1,len(temp)):
        o += temp[j]+' '
    o = o[:-1]
    relation_object.append([relation,o])

In [12]:
relation_person = []
for i in range(len(result_coref)):
    tokens = result_token[result_coref[i][1]-2] + result_token[result_coref[i][1]-1] + result_token[result_coref[i][1]]
    for obj2 in result_coref[i][0]:
        for obj3 in obj2:
            temp = word_tokenize(obj3[0])
            for obj4 in temp:
                for obj5 in tokens:
                    if obj5[0] == obj4 and obj5[3] == 'NNP' and [obj3[0],relation_object[result_coref[i][2]][0],relation_object[result_coref[i][2]][1]] not in relation_person:
                        relation_person.append([obj3[0],relation_object[result_coref[i][2]][0],relation_object[result_coref[i][2]][1]])

In [13]:
for obj in relation_person:
    print obj[2]+' is '+obj[0]+'\'s '+obj[1]

Eddard is Brandon Stark's brother
Eddard is Torrhen Karstark's brother
Benjen is his brother Benjen instead of that ragged stranger's brother
Robb is Jon's brother
Theon is Ned's son
Jaime is Robert 's queen's brother
Jaime is Cersei Lannister's brother
Ser Brynden is Lysa Arryn's brother
Ser Edmure is Lysa Arryn's son
Stannis is Joffrey's uncle
Stannis is Joffrey on his throne's uncle
Rhaegar is Rhaegar's brother
Lyanna is Lyanna's daughter
Lyanna is me , another Brandon , my father 's brother's daughter
Brandon is Lyanna's son
Brandon is me , another Brandon , my father 's brother's son
Arya is Jon Snow's sister
Arya is Jon's sister
Rhaegar is Drogo 's's brother
Rhaegar is Khal Drogo's brother
Rhaegar is Drogo's brother
Edmure is Catelyn's brother
Ser Perwyn is Ser Stevron's brother
Ser Brynden is Catelyn's uncle
Ben is Jon's uncle
Jaime is Tyrion's brother
Jaime is TYRION the Lord Commander's brother
Maege is Mormont's sister
Jaime is Tyrion Lannister's brother
Sansa is Pycelle's da